<center style="font-weight:bold;font-size:20px">wbenbihi/hourglasstensorlfow: Stacked Hourglass Network for Human Pose Estimation</center>

<center style="font-weight:bold;font-size:20px">Parsing MPII Human Pose Dataset</center>

# Setup

## Imports

In [1]:
# Standard Imports
import os
import sys
import re
import json
sys.path.append(os.path.join('..'))

In [2]:
# Specific Imports
import matplotlib.pyplot as plt
import tqdm
import pandas as pd
import numpy as np
import scipy.io
from utils import ROOT_PATH

## Global Variables

In [3]:
DATA_FOLDER = 'data'
MPII_MAT = 'mpii_human_pose_v1_u12_1.mat'

In [5]:
MAT_PATH = os.path.join(ROOT_PATH, DATA_FOLDER, MPII_MAT)

# Function definition

In [6]:
def parse_point(point):
    return {
        'point':{
            'x':point.__dict__.get('x')[0][0] if ('x' in point.__dict__) and (0 not in point.__dict__.get('x').shape) else None,
            'y':point.__dict__.get('y')[0][0] if ('y' in point.__dict__) and (0 not in point.__dict__.get('y').shape) else None,
            'id':point.__dict__.get('id')[0][0] if ('id' in point.__dict__) and (0 not in point.__dict__.get('id').shape) else None,
            'is_visible':point.__dict__.get('is_visible')[0][0] if ('is_visible' in point.__dict__) and (0 not in point.__dict__.get('is_visible').shape)  else None,
        }
    }

def parse_person(person, idx):
    return {
        'person':{
            'ridx':idx,
            'x1':person.__dict__.get('x1')[0][0] if 'x1' in (person.__dict__) and (0 not in person.__dict__.get('x1').shape) else None,
            'x2':person.__dict__.get('x2')[0][0] if 'x2' in (person.__dict__) and (0 not in person.__dict__.get('x2').shape) else None,
            'y1':person.__dict__.get('y1')[0][0] if 'y1' in (person.__dict__) and (0 not in person.__dict__.get('y1').shape) else None,
            'y2':person.__dict__.get('y2')[0][0] if 'y2' in (person.__dict__) and (0 not in person.__dict__.get('y2').shape) else None,
            'scale':person.__dict__.get('scale')[0][0] if 'scale' in (person.__dict__) and (0 not in person.__dict__.get('scale').shape) else None,
            'objpos':{
                'x':person.__dict__.get('objpos')[0][0].__dict__.get('x')[0][0] if ('objpos' in person.__dict__) and (0 not in person.__dict__.get('objpos').shape) else None,
                'y':person.__dict__.get('objpos')[0][0].__dict__.get('y')[0][0] if ('objpos' in person.__dict__) and (0 not in person.__dict__.get('objpos').shape) else None,
            },
            'points':[
                parse_point(point) for point in person.__dict__.get('annopoints')[0][0].__dict__['point'][0]
            ] if 'annopoints' in (person.__dict__) and (0 not in person.__dict__.get('annopoints').shape) else None,
        }
    }

def parse_persons(persons):
    return [
        parse_person(person, i)
        for i, person in enumerate(persons)
    ]

In [7]:
def cast_iterable(obj):
    if isinstance(obj, dict):
        return {k:cast_iterable(v) for k,v in obj.items()}
    elif isinstance(obj, list):
        return [cast_iterable(k) for k in obj]
    elif obj is None:
        return obj
    elif isinstance(obj, int) or np.issubdtype(type(obj), np.integer):
        return int(obj)
    elif isinstance(obj, float) or np.issubdtype(type(obj), np.float):
        return float(obj)
    elif isinstance(obj, str):
        return obj
    else:
        raise TypeError(f'{obj} is typed {type(obj)}')

# MPII Documentation

--------------------------------------------------------------------------- 
MPII Human Pose Dataset, Version 1.0 

Copyright 2015 Max Planck Institute for Informatics 

Licensed under the Simplified BSD License [see bsd.txt] 

--------------------------------------------------------------------------- 

We are making the annotations and the corresponding code freely available for research 
purposes. If you would like to use the dataset for any other purposes please contact 
the authors. 

## Introduction
MPII Human Pose dataset is a state of the art benchmark for evaluation
of articulated human pose estimation. The dataset includes around
**25K images** containing over **40K people** with annotated body
joints. The images were systematically collected using an established
taxonomy of every day human activities. Overall the dataset covers
**410 human activities** and each image assigned an activity
label. Each image was extracted from a YouTube video and provided with
preceding and following un-annotated frames. In addition, for the test
set we obtained richer annotations including body part occlusions and
3D torso and head orientations.

Following the best practices for the performance evaluation benchmarks
in the literature we withhold the test annotations to prevent
overfitting and tuning on the test set. We are working on an automatic
evaluation server and performance analysis tools based on rich test
set annotations.

## Citing the dataset
```
@inproceedings{andriluka14cvpr,
               author = {Mykhaylo Andriluka and Leonid Pishchulin and Peter Gehler and Schiele, Bernt}
               title = {2D Human Pose Estimation: New Benchmark and State of the Art Analysis},
               booktitle = {IEEE Conference on Computer Vision and Pattern Recognition (CVPR)},
               year = {2014},
               month = {June}
}
```

## Download

-. **Images (12.9 GB)**
   
   http://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1.tar.gz
-. **Annotations (12.5 MB)**	
   
   http://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1_u12.tar.gz
-. **Videos for each image (25 batches x 17 GB)**	

   http://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1_sequences_batch1.tar.gz
   ...
   http://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1_sequences_batch25.tar.gz
-. **Image - video mapping (239 KB)**	
   
   http://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1_sequences_keyframes.mat

## Annotation description 
Annotations are stored in a matlab structure `RELEASE` having following fields

- `.annolist(imgidx)` - annotations for image `imgidx`
  - `.image.name` - image filename
  - `.annorect(ridx)` - body annotations for a person `ridx`
      - `.x1, .y1, .x2, .y2` - coordinates of the head rectangle
      - `.scale` - person scale w.r.t. 200 px height
      - `.objpos` - rough human position in the image
      - `.annopoints.point` - person-centric body joint annotations
        - `.x, .y` - coordinates of a joint
        - `id` 
            - joint id [//]: # "(0 - r ankle, 1 - r knee, 2 - r hip, 3 - l hip, 4 - l knee, 5 - l ankle, 6 - pelvis, 7 - thorax, 8 - upper neck, 9 - head top, 10 - r wrist, 10 - r wrist, 12 - r shoulder, 13 - l shoulder, 14 - l elbow, 15 - l wrist)"
        - `is_visible` - joint visibility
  - `.vidx` - video index in `video_list`
  - `.frame_sec` - image position in video, in seconds
 
- `img_train(imgidx)` - training/testing image assignment 
- `single_person(imgidx)` - contains rectangle id `ridx` of *sufficiently separated* individuals
- `act(imgidx)` - activity/category label for image `imgidx`
  - `act_name` - activity name
  - `cat_name` - category name
  - `act_id` - activity id
- `video_list(videoidx)` - specifies video id as is provided by YouTube. To watch video on youtube go to https://www.youtube.com/watch?v=video_list(videoidx) 

## Browsing the dataset
- Please use our online tool for browsing the data
http://human-pose.mpi-inf.mpg.de/#dataset
- Red rectangles mark testing images

## References
- **2D Human Pose Estimation: New Benchmark and State of the Art Analysis.**

  Mykhaylo Andriluka, Leonid Pishchulin, Peter Gehler and Bernt Schiele. 

  IEEE CVPR'14
- **Fine-grained Activity Recognition with Holistic and Pose based Features.**

  Leonid Pishchulin, Mykhaylo Andriluka and Bernt Schiele.

  GCPR'14

## Contact
You can reach us via `<lastname>@mpi-inf.mpg.de`
We are looking forward to your feedback. If you have any questions related to the dataset please let us know.


# Main Code

Since MPII Human Pose Dataset labels are recorded in a MATLAB .mat file, we need to parse it to a clean pandas DataFrame. This format is heavily nested and needs a little bit of exploration to parse it completely

In [8]:
# Load .mat file
mat = scipy.io.loadmat(MAT_PATH, struct_as_record=False)
release_mat = mat['RELEASE'][0][0]

We check if the `fieldnames` are correct

In [8]:
release_mat._fieldnames

['annolist', 'img_train', 'version', 'single_person', 'act', 'video_list']

In [9]:
# Accessing coordinates X of Point 0 from Person 0 in Image 4
release_mat.__dict__['annolist'][0][4].__dict__['annorect'][0][0].__dict__['annopoints'][0][0].__dict__['point'][0][0].x

array([[610]], dtype=uint16)

In [10]:
# Train/Test Label
img_train = release_mat.__dict__.get('img_train')[0]

In [11]:
# List of Videos
video_list = release_mat.__dict__.get('video_list')[0]
video_list_json = [{'video': {'videoidx':i, 'video_list':item[0]}} for i, item in enumerate(video_list)]

In [12]:
mpii_version = release_mat.__dict__.get('version')[0]
annolist = release_mat.__dict__.get('annolist')[0]
single_person = release_mat.__dict__.get('single_person')
act = release_mat.__dict__.get('act')

## Handle act

In [13]:
len(act)

24987

In [14]:
act[4][0]._fieldnames

['cat_name', 'act_name', 'act_id']

In [15]:
act_json = [
    {
        'act':{
            'imgidx':i,
            'cat_name':elem[0].__dict__.get('cat_name')[0] if len(elem[0].__dict__.get('cat_name')) else None,
            'act_name':elem[0].__dict__.get('act_name')[0].split(', ') if len(elem[0].__dict__.get('act_name')) else None,
            'act_id':elem[0].__dict__.get('act_id')[0][0]
        }
    } 
    for i, elem in enumerate(act)
]

## Handle single_person

In [16]:
len(single_person)

24987

In [17]:
single_person_json = [
    {
        'single_person':{
            'imgidx':i,
            'ridx': [elm[0] for elm in item[0]] if 0 not in item[0].shape else None
        }
    }
    for i, item in enumerate(single_person)
]

## Handle Annopoints

In [18]:
annolist[0]._fieldnames

['image', 'annorect', 'frame_sec', 'vididx']

In [19]:
annolist_parse_head = [
    {
        'annopoint':{
            'imgidx':i,
            'image':item.__dict__.get('image')[0][0].__dict__.get('name')[0],
            'annorect':item.__dict__.get('annorect'),
            'frame_sec':item.__dict__.get('frame_sec')[0] if 0 not in item.__dict__.get('frame_sec').shape else None,
            'vididx':item.__dict__.get('vididx')[0][0] if 0 not in item.__dict__.get('vididx').shape else None,
        }
    }
    for i, item in enumerate(annolist)
]

### Sample

In [20]:
# Sample with raw parsing
annolist_parse_head[0:5]

[{'annopoint': {'imgidx': 0,
   'image': '037454012.jpg',
   'annorect': array([[<scipy.io.matlab.mio5_params.mat_struct object at 0x120113990>]],
         dtype=object),
   'frame_sec': None,
   'vididx': None}},
 {'annopoint': {'imgidx': 1,
   'image': '095071431.jpg',
   'annorect': array([[<scipy.io.matlab.mio5_params.mat_struct object at 0x1200e64d0>,
         dtype=object),
   'frame_sec': None,
   'vididx': None}},
 {'annopoint': {'imgidx': 2,
   'image': '073199394.jpg',
   'annorect': array([[<scipy.io.matlab.mio5_params.mat_struct object at 0x1200e6250>]],
         dtype=object),
   'frame_sec': None,
   'vididx': None}},
 {'annopoint': {'imgidx': 3,
   'image': '059865848.jpg',
   'annorect': array([[<scipy.io.matlab.mio5_params.mat_struct object at 0x1200e6750>]],
         dtype=object),
   'frame_sec': None,
   'vididx': None}},
 {'annopoint': {'imgidx': 4,
   'image': '015601864.jpg',
   'annorect': array([[<scipy.io.matlab.mio5_params.mat_struct object at 0x1200e6a90>,
 

In [21]:
IDX = 4
annolist_parse_head[IDX]['annopoint']['annorect'][0]

array([<scipy.io.matlab.mio5_params.mat_struct object at 0x1200e6a90>,
      dtype=object)

In [22]:
IDPERS = 1
annolist_parse_head[IDX]['annopoint']['annorect'][0][IDPERS]._fieldnames

['x1', 'y1', 'x2', 'y2', 'annopoints', 'scale', 'objpos']

In [23]:
(annolist_parse_head[IDX]['annopoint']['annorect'][0][IDPERS].__dict__.get('x1')[0][0],
annolist_parse_head[IDX]['annopoint']['annorect'][0][IDPERS].__dict__.get('x2')[0][0],
annolist_parse_head[IDX]['annopoint']['annorect'][0][IDPERS].__dict__.get('y1')[0][0],
annolist_parse_head[IDX]['annopoint']['annorect'][0][IDPERS].__dict__.get('y2')[0][0])

(841, 902, 145, 228)

In [24]:
annolist_parse_head[IDX]['annopoint']['annorect'][0][IDPERS].__dict__.get('scale')[0][0]

2.472116502109073

In [25]:
(annolist_parse_head[IDX]['annopoint']['annorect'][0][IDPERS].__dict__.get('objpos')[0][0].__dict__.get('x')[0][0],
annolist_parse_head[IDX]['annopoint']['annorect'][0][IDPERS].__dict__.get('objpos')[0][0].__dict__.get('y')[0][0])

(952, 222)

In [26]:
annolist_parse_head[IDX]['annopoint']['annorect'][0][IDPERS].__dict__.get('annopoints')[0][0].__dict__['point'][0]

array([<scipy.io.matlab.mio5_params.mat_struct object at 0x1200d0b10>,
      dtype=object)

In [27]:
annolist_parse_head[IDX]['annopoint']['annorect'][0][IDPERS].__dict__.get('annopoints')[0][0].__dict__['point'][0][0]._fieldnames

['x', 'y', 'id', 'is_visible']

In [28]:
(annolist_parse_head[IDX]['annopoint']['annorect'][0][IDPERS].__dict__.get('annopoints')[0][0].__dict__['point'][0][0].__dict__.get('x')[0][0],
annolist_parse_head[IDX]['annopoint']['annorect'][0][IDPERS].__dict__.get('annopoints')[0][0].__dict__['point'][0][0].__dict__.get('y')[0][0],
annolist_parse_head[IDX]['annopoint']['annorect'][0][IDPERS].__dict__.get('annopoints')[0][0].__dict__['point'][0][0].__dict__.get('id')[0][0],
annolist_parse_head[IDX]['annopoint']['annorect'][0][IDPERS].__dict__.get('annopoints')[0][0].__dict__['point'][0][0].__dict__.get('is_visible')[0][0])

(979, 221, 6, 0)

### Coarse Parsing

In [29]:
annolist_parsed = [
    {
        'annopoint':{
            'imgidx':i,
            'image':item.__dict__.get('image')[0][0].__dict__.get('name')[0],
            'annorect':parse_persons(item.__dict__.get('annorect')[0]) if 0 not in item.__dict__.get('annorect').shape else None,
            'frame_sec':item.__dict__.get('frame_sec')[0][0] if 0 not in item.__dict__.get('frame_sec').shape else None,
            'vididx':item.__dict__.get('vididx')[0][0] if 0 not in item.__dict__.get('vididx').shape else None,
        }
    }
    for i, item in enumerate(annolist)
]

In [30]:
annolist_parsed[:20]

[{'annopoint': {'imgidx': 0,
   'image': '037454012.jpg',
   'annorect': [{'person': {'ridx': 0,
      'x1': None,
      'x2': None,
      'y1': None,
      'y2': None,
      'scale': 3.8807339512004684,
      'objpos': {'x': 601, 'y': 380},
      'points': None}}],
   'frame_sec': None,
   'vididx': None}},
 {'annopoint': {'imgidx': 1,
   'image': '095071431.jpg',
   'annorect': [{'person': {'ridx': 0,
      'x1': None,
      'x2': None,
      'y1': None,
      'y2': None,
      'scale': 8.078166128521993,
      'objpos': {'x': 881, 'y': 394},
      'points': None}},
    {'person': {'ridx': 1,
      'x1': None,
      'x2': None,
      'y1': None,
      'y2': None,
      'scale': 8.90412937911394,
      'objpos': {'x': 338, 'y': 210},
      'points': None}}],
   'frame_sec': None,
   'vididx': None}},
 {'annopoint': {'imgidx': 2,
   'image': '073199394.jpg',
   'annorect': [{'person': {'ridx': 0,
      'x1': None,
      'x2': None,
      'y1': None,
      'y2': None,
      'scale': 4.3

## To JSON

In [31]:
json_to_save = [
    (cast_iterable(annolist_parsed), 'annolist.json'),
    (cast_iterable(single_person_json), 'single_person.json'),
    (cast_iterable(video_list_json), 'video_list.json'),
    (cast_iterable(act_json), 'act.json'),
    (cast_iterable(img_train.tolist()), 'img_train.json'),
]

/Users/walid/.local/share/virtualenvs/hourglasstensorlfow-kuwAf2CD/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  # Remove the CWD from sys.path while we load stuff.


In [32]:
for d, p in json_to_save:
    with open(os.path.join(ROOT_PATH,DATA_FOLDER, p), 'w') as f:
        json.dump(d, f)

## Read JSON

In [33]:
mpii_data = {}
for j in ['annolist', 'act', 'img_train', 'single_person', 'video_list']:
    with open(os.path.join(ROOT_PATH,DATA_FOLDER, f'{j}.json'), 'r') as f:
        mpii_data[j] = json.load(f)

In [34]:
img_df = pd.DataFrame.from_dict([ann['annopoint'] for ann in mpii_data['annolist']])#.set_index('imgidx')
img_df['is_train'] = mpii_data['img_train']
img_df['frame_sec'] = img_df['frame_sec'].fillna(-1).astype(int)
img_df['vididx'] = img_df['vididx'].fillna(-1).astype(int)
_ = img_df.apply(lambda x: [person['person'].update({'imgidx': x.imgidx}) for person in x.annorect] if x.annorect is not None else 0, axis=1)
img_df

,imgidx,image,annorect,frame_sec,vididx,is_train
0,0,037454012.jpg,"[{'person': {'ridx': 0, 'x1': None, 'x2': None...",-1,-1,0
1,1,095071431.jpg,"[{'person': {'ridx': 0, 'x1': None, 'x2': None...",-1,-1,0
2,2,073199394.jpg,"[{'person': {'ridx': 0, 'x1': None, 'x2': None...",-1,-1,0
3,3,059865848.jpg,"[{'person': {'ridx': 0, 'x1': None, 'x2': None...",-1,-1,0
4,4,015601864.jpg,"[{'person': {'ridx': 0, 'x1': 627, 'x2': 706, ...",11,1660,1
...,...,...,...,...,...,...
24982,24982,072772110.jpg,"[{'person': {'ridx': 0, 'x1': 506, 'x2': 548, ...",121,1587,1
24983,24983,039361034.jpg,"[{'person': {'ridx': 0, 'x1': 406, 'x2': 449, ...",135,793,1
24984,24984,084761779.jpg,"[{'person': {'ridx': 0, 'x1': 287, 'x2': 324, ...",158,793,1
24985,24985,092025825.jpg,"[{'person': {'ridx': 0, 'x1': None, 'x2': None...",-1,-1,0


In [35]:
persons = img_df.query('annorect.notna()').annorect.sum()
person_df = pd.DataFrame([person['person'] for person in persons])
person_df = person_df.assign(
    objpos_x = person_df.objpos.map(lambda x: x['x']),
    objpos_y = person_df.objpos.map(lambda x: x['y'])
)
person_df

,ridx,x1,x2,y1,y2,scale,objpos,points,imgidx,objpos_x,objpos_y
0,0,NaN,NaN,NaN,NaN,3.880734,"{'x': 601, 'y': 380}",None,0,601.0,380.0
1,0,NaN,NaN,NaN,NaN,8.078166,"{'x': 881, 'y': 394}",None,1,881.0,394.0
2,1,NaN,NaN,NaN,NaN,8.904129,"{'x': 338, 'y': 210}",None,1,338.0,210.0
3,0,NaN,NaN,NaN,NaN,4.326662,"{'x': 619, 'y': 350}",None,2,619.0,350.0
4,0,NaN,NaN,NaN,NaN,4.928480,"{'x': 684, 'y': 309}",None,3,684.0,309.0
...,...,...,...,...,...,...,...,...,...,...,...
40934,4,0.0,76.0,135.0,216.0,2.665729,"{'x': 62, 'y': 338}","[{'point': {'x': 68, 'y': 416, 'id': 6, 'is_vi...",24984,62.0,338.0
40935,0,NaN,NaN,NaN,NaN,9.880337,"{'x': 1224, 'y': 553}",None,24985,1224.0,553.0
40936,1,NaN,NaN,NaN,NaN,8.619509,"{'x': 473, 'y': 587}",None,24985,473.0,587.0
40937,0,NaN,NaN,NaN,NaN,8.844122,"{'x': 1277, 'y': 681}",None,24986,1277.0,681.0


In [36]:
joint_dict = {}
for idx_joint in range(16):
    joint_dict.update(
        {
            f'joint_{idx_joint}':person_df.points.apply(
                lambda x : [
                    joint['point'] 
                    for joint in x 
                    if joint['point']['id'] == idx_joint
                ] if x is not None else None
            ).str[0]
        }
    )
person_df = person_df.assign(**joint_dict)
person_df

,ridx,x1,x2,y1,y2,scale,objpos,points,imgidx,objpos_x,...,joint_6,joint_7,joint_8,joint_9,joint_10,joint_11,joint_12,joint_13,joint_14,joint_15
0,0,NaN,NaN,NaN,NaN,3.880734,"{'x': 601, 'y': 380}",None,0,601.0,...,None,None,None,None,None,None,None,None,None,None
1,0,NaN,NaN,NaN,NaN,8.078166,"{'x': 881, 'y': 394}",None,1,881.0,...,None,None,None,None,None,None,None,None,None,None
2,1,NaN,NaN,NaN,NaN,8.904129,"{'x': 338, 'y': 210}",None,1,338.0,...,None,None,None,None,None,None,None,None,None,None
3,0,NaN,NaN,NaN,NaN,4.326662,"{'x': 619, 'y': 350}",None,2,619.0,...,None,None,None,None,None,None,None,None,None,None
4,0,NaN,NaN,NaN,NaN,4.928480,"{'x': 684, 'y': 309}",None,3,684.0,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40934,4,0.0,76.0,135.0,216.0,2.665729,"{'x': 62, 'y': 338}","[{'point': {'x': 68, 'y': 416, 'id': 6, 'is_vi...",24984,62.0,...,"{'x': 68, 'y': 416, 'id': 6, 'is_visible': 1}",NaN,NaN,NaN,"{'x': 167, 'y': 342, 'id': 10, 'is_visible': 1}","{'x': 132, 'y': 316, 'id': 11, 'is_visible': 1}","{'x': 91, 'y': 237, 'id': 12, 'is_visible': 1}",NaN,NaN,NaN
40935,0,NaN,NaN,NaN,NaN,9.880337,"{'x': 1224, 'y': 553}",None,24985,1224.0,...,None,None,None,None,None,None,None,None,None,None
40936,1,NaN,NaN,NaN,NaN,8.619509,"{'x': 473, 'y': 587}",None,24985,473.0,...,None,None,None,None,None,None,None,None,None,None
40937,0,NaN,NaN,NaN,NaN,8.844122,"{'x': 1277, 'y': 681}",None,24986,1277.0,...,None,None,None,None,None,None,None,None,None,None


In [37]:
for joint in range(16):
    joint_dict = {}
    series = person_df[f'joint_{joint}']
    joint_dict = {
        f'joint_{joint}_x':series.str['x'],
        f'joint_{joint}_y':series.str['y'],
        f'joint_{joint}_id':series.str['id'],
        f'joint_{joint}_is_visible':series.str['is_visible'],
    }
    person_df = person_df.assign(**joint_dict)
person_df

,ridx,x1,x2,y1,y2,scale,objpos,points,imgidx,objpos_x,...,joint_13_id,joint_13_is_visible,joint_14_x,joint_14_y,joint_14_id,joint_14_is_visible,joint_15_x,joint_15_y,joint_15_id,joint_15_is_visible
0,0,NaN,NaN,NaN,NaN,3.880734,"{'x': 601, 'y': 380}",None,0,601.0,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
1,0,NaN,NaN,NaN,NaN,8.078166,"{'x': 881, 'y': 394}",None,1,881.0,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
2,1,NaN,NaN,NaN,NaN,8.904129,"{'x': 338, 'y': 210}",None,1,338.0,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
3,0,NaN,NaN,NaN,NaN,4.326662,"{'x': 619, 'y': 350}",None,2,619.0,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
4,0,NaN,NaN,NaN,NaN,4.928480,"{'x': 684, 'y': 309}",None,3,684.0,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40934,4,0.0,76.0,135.0,216.0,2.665729,"{'x': 62, 'y': 338}","[{'point': {'x': 68, 'y': 416, 'id': 6, 'is_vi...",24984,62.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40935,0,NaN,NaN,NaN,NaN,9.880337,"{'x': 1224, 'y': 553}",None,24985,1224.0,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
40936,1,NaN,NaN,NaN,NaN,8.619509,"{'x': 473, 'y': 587}",None,24985,473.0,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
40937,0,NaN,NaN,NaN,NaN,8.844122,"{'x': 1277, 'y': 681}",None,24986,1277.0,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None


In [38]:
HEAD_COLUMNS = ['imgidx', 'ridx','x1', 'x2', 'y1', 'y2', 'scale', 'objpos_x', 'objpos_y']
JOINT_COLUMNS = [col for col in person_df.columns if re.match("joint_[0-9]*_(x|y|id|is_visible)", col)]
single_person_df = person_df[HEAD_COLUMNS+JOINT_COLUMNS]
single_person_df

,imgidx,ridx,x1,x2,y1,y2,scale,objpos_x,objpos_y,joint_0_x,...,joint_13_id,joint_13_is_visible,joint_14_x,joint_14_y,joint_14_id,joint_14_is_visible,joint_15_x,joint_15_y,joint_15_id,joint_15_is_visible
0,0,0,NaN,NaN,NaN,NaN,3.880734,601.0,380.0,NaN,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
1,1,0,NaN,NaN,NaN,NaN,8.078166,881.0,394.0,NaN,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
2,1,1,NaN,NaN,NaN,NaN,8.904129,338.0,210.0,NaN,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
3,2,0,NaN,NaN,NaN,NaN,4.326662,619.0,350.0,NaN,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
4,3,0,NaN,NaN,NaN,NaN,4.928480,684.0,309.0,NaN,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40934,24984,4,0.0,76.0,135.0,216.0,2.665729,62.0,338.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40935,24985,0,NaN,NaN,NaN,NaN,9.880337,1224.0,553.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40936,24985,1,NaN,NaN,NaN,NaN,8.619509,473.0,587.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40937,24986,0,NaN,NaN,NaN,NaN,8.844122,1277.0,681.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
mpii_dataset = img_df[
    ['imgidx', 'image', 'frame_sec', 'vididx', 'is_train']
].merge(
    single_person_df,
    on='imgidx',
    how='inner'
)
mpii_dataset

,imgidx,image,frame_sec,vididx,is_train,ridx,x1,x2,y1,y2,...,joint_13_id,joint_13_is_visible,joint_14_x,joint_14_y,joint_14_id,joint_14_is_visible,joint_15_x,joint_15_y,joint_15_id,joint_15_is_visible
0,0,037454012.jpg,-1,-1,0,0,NaN,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
1,1,095071431.jpg,-1,-1,0,0,NaN,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
2,1,095071431.jpg,-1,-1,0,1,NaN,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
3,2,073199394.jpg,-1,-1,0,0,NaN,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
4,3,059865848.jpg,-1,-1,0,0,NaN,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40934,24984,084761779.jpg,158,793,1,4,0.0,76.0,135.0,216.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40935,24985,092025825.jpg,-1,-1,0,0,NaN,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
40936,24985,092025825.jpg,-1,-1,0,1,NaN,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
40937,24986,053686627.jpg,-1,-1,0,0,NaN,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None


In [40]:
act_df = pd.DataFrame([act['act'] for act in mpii_data['act']])
act_df = act_df.assign(
    act_name = act_df.act_name.str.join(", ")
)
act_df

,imgidx,cat_name,act_name,act_id
0,0,None,None,-1
1,1,None,None,-1
2,2,None,None,-1
3,3,None,None,-1
4,4,sports,curling,1
...,...,...,...,...
24982,24982,transportation,pushing car,972
24983,24983,transportation,pushing car,972
24984,24984,transportation,pushing car,972
24985,24985,None,None,-1


In [41]:
sp_df = pd.DataFrame([sp['single_person'] for sp in mpii_data['single_person']])
sp_df

,imgidx,ridx
0,0,[1]
1,1,None
2,2,[1]
3,3,[1]
4,4,"[1, 2]"
...,...,...
24982,24982,[1]
24983,24983,[1]
24984,24984,[5]
24985,24985,None


In [42]:
video_df = pd.DataFrame([vl['video'] for vl in mpii_data['video_list']]).rename(columns={'videoidx':'vididx'})
video_df.vididx += 1
video_df

,vididx,video_list
0,1,-08Vnk8XONY
1,2,-0YdHqlKV8w
2,3,-1nf9pRuuZw
3,4,-2cO6M_ssKo
4,5,-2lo5Iz3gRY
...,...,...
2816,2817,zufm7uGFkuY
2817,2818,zuhzWKbRq6s
2818,2819,zvMWkSAcSVc
2819,2820,zwqQrtD2L84


In [43]:
mpii_dataset = mpii_dataset.merge(
    video_df,
    on='vididx',
    how='left'
).merge(
    act_df,
    on='imgidx',
    how='left'
)

In [44]:
mpii_dataset.to_csv(os.path.join(ROOT_PATH, DATA_FOLDER, 'MPII_DATASET_LABELS.csv'), sep=';', index=False)